In [1]:
import glob
import json
import os

import anthropic

In [ ]:
conversations = []

paths = glob.glob('./resources/aihub_sns/297.SNS 데이터 고도화/01-1.정식개방데이터/Validation/02.라벨링데이터/VL/*json')
target_count = 20
count = 0

for path in paths:
    with open(path, 'r') as f:
        conv_dict = json.load(f)
        if conv_dict['header']['dialogueInfo']['numberOfParticipants'] == 2:
            if conv_dict['header']['dialogueInfo']['numberOfUtterances'] > 30:
                conv_list = []
                for d in conv_dict['body']:
                    conv_list.append(d['participantID'] + ': ' + d['utterance'])
                if conv_list[0] == conv_list[1]:
                    print('Repeated Conversations')
                    continue
                conv_text = '\n'.join(conv_list)
                conversations.append(conv_text)
                count += 1
                if count == target_count:
                    break

target_count = 30
count = 0

for path in paths:
    with open(path, 'r') as f:
        conv_dict = json.load(f)
        if conv_dict['header']['dialogueInfo']['numberOfParticipants'] > 2:
            if conv_dict['header']['dialogueInfo']['numberOfUtterances'] > 30:
                conv_list = []
                for d in conv_dict['body']:
                    conv_list.append(d['participantID'] + ': ' + d['utterance'])
                if conv_list[0] == conv_list[1]:           
                    continue
                conv_text = '\n'.join(conv_list)
                conversations.append(conv_text)
                count += 1
                if count == target_count:
                    break

count

In [ ]:
prompt = f"""You are given a conversation among three users below. Imagine what the users discussed prior to the conversation.
Please write the previous content, strictly following the tone and format of given conversation, with at least 3000 characters.

{conversations[-2]}
"""

In [ ]:
print(conversations[-2])

In [ ]:
client = anthropic.Anthropic(
    api_key=os.environ['ANTHROPIC_API_KEY']
)

message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=4096,
    temperature=0.0,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

message.usage  # API 호출 비용 대략 50원

In [ ]:
message.content[0].text.split('\n\n')

In [ ]:
lst = message.content[0].text.split('\n\n')[1:-1]
for l in lst[:25]:
    print(l)

In [ ]:
for l in lst[25:50]:
    print(l)

In [ ]:
print(len(message.content[0].text.split('\n\n')[1]))

In [ ]:
lst = message.content[0].text.split('\n\n')[1:-1]
lst

In [ ]:
import pickle

with open('./resources/conv_long.pickle', 'wb') as f:
    pickle.dump(lst, f)